In [1]:
import numpy as np
import os
from scipy.stats import norm
import math
import scipy.optimize as opt
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from pandas.tseries.offsets import DateOffset
from dateutil.relativedelta import relativedelta
from scipy.optimize import fsolve, root
import datetime
import copy
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm


In [47]:
import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [48]:
lambdas_5 = pd.Series(data = np.array([0.34850838, 0.34850838, 0.38474649, 0.39183218, 0.38349713]), 
                    index = [0,1,2,3,4])

lambdas_10 = pd.Series(data = np.array([0.3292204326569119, 0.3292204326569119, 0.3527551663944037, 0.3521045801361138, 0.34228387066529303, 0.3303977808906562, 0.32180652778597957, 0.3163107875080656, 0.309829093946949, 0.29495363523738527]), 
                    index = [0,1,2,3,4,5,6,7,8,9])

lambdas_15 = pd.Series(data = np.array([0.31019157,0.31019157, 0.32745481, 0.32667033, 0.31896721, 0.30683361, 0.29715606, 0.28673488, 0.27939385, 0.27297822, 0.26892379, 0.26938024, 0.26940732, 0.27254794, 0.26907595]), 
                    index = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14])

lambdas_20 = pd.Series(data = np.array([0.3056834 ,0.3056834 , 0.32158634, 0.32035852, 0.31258137, 0.30140343, 0.29393848, 0.28617513, 0.27892284, 0.27405485, 0.26961037, 0.26910762, 0.26992212, 0.27178778, 0.27518986, 0.28242963, 0.28726692, 0.2910441 , 0.29531973, 0.29225949]), 
                    index = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])

In [57]:
# latex_width = 6.69423 inches
w = 6.69423 * 7.6/10
print(w)
golden_ratio = (5**.5 - 1) / 2
h = 4.685961000000001* golden_ratio
h

5.0876148


2.8960831679564465

In [50]:
fig, ax = plt.subplots(1,1, figsize=(w,h))

ax.set_ylim(0.15,0.5)
ax.set_yticks(np.arange(0.15, 0.55,0.05))
ax.set_xlim(0, 19)
ax.set_xticks(np.arange(0,20,1))
ax.set_xlabel('Option expiry (Years)')
ax.set_title('$\\lambda(t)$')

plt.step(lambdas_5.index, lambdas_5, label = '5nc1', color = 'b')
plt.step(lambdas_10.index, lambdas_10, label = '10nc1', color = 'r')
plt.step(lambdas_15.index, lambdas_15, label = '15nc1', color = 'g')
plt.step(lambdas_20.index, lambdas_20, label = '20nc1', color = '#ff7f0e')

leg = plt.legend(frameon=False, loc = 'upper right', fontsize=9)
ax.grid(ls = (0,(1,2)))
fig.tight_layout()
plt.savefig('C:/Users/Mikkel/OneDrive/CBS/CBS - FIN/THESIS/Main/python/billeder/lambdas_2_coterm.pgf')
plt.show()

C:\Users\Mikkel\AppData\Local\Temp\ipykernel_11488\803121567.py:19: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## Calibration 3

In [63]:
# 5nc1
lambd_calibrated_5 = pd.Series(data = np.array([0.35376202,0.35376202, 0.38292605, 0.38977147, 0.37911044]), 
                    index = [0,1,2,3,4])

b_calibrated_5 = pd.Series(data = np.array([0.00475537,0.00475537, 0.12264038, 0.1412259 , 0.17313736]), 
                    index = [0,1,2,3,4])

eta_calibrated_5 = pd.Series(data = np.array([1.72602727,1.72602727, 1.43924873, 1.43655247, 1.37274629]), 
                    index = [0,1,2,3,4])


# 10nc1
lambd_calibrated_10 = pd.Series(data = np.array([0.33188365,0.33188365, 0.35257935, 0.35325137, 0.34156719, 0.32450657, 0.31750145, 0.31195555, 0.30649471, 0.29229506]), 
                    index = [0,1,2,3,4,5,6,7,8,9])

b_calibrated_10 = pd.Series(data = np.array([0.1728704,0.1728704, 0.18118124, 0.18003633, 0.19062064, 0.21062688, 0.21702723, 0.1970315 , 0.23731793, 0.24521783]), 
                    index = [0,1,2,3,4,5,6,7,8,9])

eta_calibrated_10 = pd.Series(data = np.array([1.60404658,1.60404658, 1.48596508, 1.52336611, 1.47089641, 1.3110149, 1.3522574 , 1.34686759, 1.3679911, 1.38158914]), 
                    index = [0,1,2,3,4,5,6,7,8,9])


# 15nc1
lambd_calibrated_15 = pd.Series(data = np.array([0.31252499,0.31252499, 0.32716261, 0.32804561, 0.31906521, 0.30234794, 0.29556366, 0.28708199, 0.28092718, 0.27477928, 0.2700827, 0.27082702, 0.27060634, 0.27382635, 0.27106085]), 
                    index = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14])

b_calibrated_15 = pd.Series(data = np.array([0.20914441,0.20914441, 0.21395428, 0.20897396, 0.2004646 , 0.19038863, 0.19244539, 0.19595601, 0.19932191, 0.20335474, 0.20738039, 0.23204294, 0.23611073, 0.2660578 , 0.28114119]), 
                    index = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14])

eta_calibrated_15 = pd.Series(data = np.array([1.57820697,1.57820697, 1.46156327, 1.5123661 , 1.47471207, 1.32609028, 1.41845365, 1.48493632, 1.53012501, 1.54337588, 1.51934625, 1.52744571, 1.51632786, 1.50961771, 1.5378323]), 
                    index = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14])


# 20nc1
lambd_calibrated_20 = pd.Series(data = np.array([0.30745666,0.30745666, 0.31998112, 0.32030697, 0.3116415 , 0.29668979, 0.2922619 , 0.28687346, 0.28136226, 0.27681879, 0.27377777, 0.27391536, 0.2747742 , 0.27658182, 0.27946444, 0.2856533, 0.29013932, 0.29335701, 0.29714964, 0.29369573]), 
                    index = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])

b_calibrated_20 = pd.Series(data = np.array([0.16742349,0.16742349, 0.17048661, 0.1745532 , 0.17584704, 0.17523996, 0.17366574, 0.17042437, 0.16495084, 0.12140896, 0.14615985, 0.1606869 , 0.16399058, 0.17519446, 0.17904733, 0.17411346, 0.1930814 , 0.21002483, 0.22534451, 0.22696482]), 
                    index = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])

eta_calibrated_20 = pd.Series(data = np.array([1.5605828 ,1.5605828 , 1.42328526, 1.47545642, 1.44735756, 1.31879797, 1.41867505, 1.50233187, 1.57036451, 1.59282986, 1.65338718, 1.68384853, 1.69073158, 1.69040665, 1.66943619, 1.62360644, 1.60202822, 1.56960267, 1.53886673, 1.52000955]), 
                    index = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])

In [59]:
fig, ax = plt.subplots(1,1, figsize=(w,h))

ax.set_ylim(0.15,0.5)
ax.set_yticks(np.arange(0.15, 0.55,0.05))
ax.set_xlim(0, 19)
ax.set_xticks(np.arange(0,20,1))
ax.set_xlabel('Option expiry (Years)')
ax.set_title('$\\lambda(t)$')

plt.step(lambd_calibrated_5.index, lambd_calibrated_5, label = '5nc1', color = 'b')
plt.step(lambd_calibrated_10.index, lambd_calibrated_10, label = '10nc1', color = 'r')
plt.step(lambd_calibrated_15.index, lambd_calibrated_15, label = '15nc1', color = 'g')
plt.step(lambd_calibrated_20.index, lambd_calibrated_20, label = '20nc1', color = '#ff7f0e')

leg = plt.legend(frameon=False, loc = 'upper right', fontsize=9)
ax.grid(ls = (0,(1,2)))
fig.tight_layout()
plt.savefig('C:/Users/Mikkel/OneDrive/CBS/CBS - FIN/THESIS/Main/python/billeder/lambdas_3_coterm.pgf')
plt.show()

C:\Users\Mikkel\AppData\Local\Temp\ipykernel_11488\2615717067.py:19: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [64]:
# latex_width = 6.69423 inches
# latex_height = 
w = 6.69423 * 5.21/10
golden_ratio = (5**.5 - 1) / 2
h = 6.69423 * 5/6 * golden_ratio
h

3.4477180570910075

In [65]:
fig, ax = plt.subplots(1,1, figsize=(w,h))

ax.set_ylim(0,0.5)
ax.set_yticks(np.arange(0,0.6,0.1))
ax.set_xlim(0, 19)
ax.set_xticks(np.arange(0,20,2))
ax.set_xlabel('Option expiry (Years)')
ax.set_title('$b(t)$')

plt.step(b_calibrated_5.index, b_calibrated_5, label = '5nc1', color = 'b')
plt.step(b_calibrated_10.index, b_calibrated_10, label = '10nc1', color = 'r')
plt.step(b_calibrated_15.index, b_calibrated_15, label = '15nc1', color = 'g')
plt.step(b_calibrated_20.index, b_calibrated_20, label = '20nc1', color = '#ff7f0e')

plt.legend(frameon=False, loc = 'upper right', fontsize=9)
ax.grid(ls = (0,(1,2)))
fig.tight_layout()
plt.savefig('C:/Users/Mikkel/OneDrive/CBS/CBS - FIN/THESIS/Main/python/billeder/bs_3_coterm.pgf')
plt.show()

C:\Users\Mikkel\AppData\Local\Temp\ipykernel_11488\3133531371.py:19: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [66]:
fig, ax = plt.subplots(1,1, figsize=(w,h))

ax.set_ylim(1,2)
ax.set_yticks(np.arange(1,2.25,0.25))
ax.set_xlim(0, 19)
ax.set_xticks(np.arange(0,20,2))
ax.set_xlabel('Option expiry (Years)')
ax.set_title('$\\eta(t)$')

plt.step(eta_calibrated_5.index, eta_calibrated_5, label = '5nc1', color = 'b')
plt.step(eta_calibrated_10.index, eta_calibrated_10, label = '10nc1', color = 'r')
plt.step(eta_calibrated_15.index, eta_calibrated_15, label = '15nc1', color = 'g')
plt.step(eta_calibrated_20.index, eta_calibrated_20, label = '20nc1', color = '#ff7f0e')

plt.legend(frameon=False, loc = 'upper right', fontsize=8)
ax.grid(ls = (0,(1,2)))
fig.tight_layout()
plt.savefig('C:/Users/Mikkel/OneDrive/CBS/CBS - FIN/THESIS/Main/python/billeder/etas_3_coterm.pgf')
plt.show()

C:\Users\Mikkel\AppData\Local\Temp\ipykernel_11488\2724851323.py:19: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
